## Battle of Neighbourhood

## Week 1 

## 1 Introduction to problem and Idea
People living in Singapore,	within walking distance to Downtown "Telok Ayer  MRT metro station" . They enjoy great venues and attractions, such as international  cuisine, entertainment and shopping. My brother have an oﬀer to move to work to Manhattan NY  and My brother would like to move if I can find a place to live similar with similar venues.

Problem to be resolved:
How to find an apartment in Manhattan with the following conditions:
Apartment with min 2 bedrooms
Monthly rent not to exceed US$7000/month
Located within walking distance (<=1.0 mile, 1.6 km) from a subway metro station in  Manhattan
Venues and amenities as in my current residence.

Interested Audience
I believe the methodology, tools and strategy used in this project is relevant for a person  or entity considering moving to a major city in US, Europe or Asia.	Europe, US or Asia,  Likewise, it can be helpful approach to explore the opening of a new business.	The use  of FourSquare data and mapping techniques combined with data analysis will help  resolve the key questions arisen. Lastly, this project is a good practical case for a person  developing Data Science skills.

## codes and library installed to extract data 

In [1]:
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.colors as colors
import seaborn as sns
from sklearn.cluster import KMeans

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


In [3]:
CLIENT_ID = 'YASVWXRHKI3UTPILR3URBA2BJYJKGKCRYIEABDUXCSI5K5IF' 
CLIENT_SECRET = '32VFUXYBPBYD2MKLBMEMVVC2VMYGRQ0XWMRNXQ0BSS01UCWC'
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YASVWXRHKI3UTPILR3URBA2BJYJKGKCRYIEABDUXCSI5K5IF
CLIENT_SECRET:32VFUXYBPBYD2MKLBMEMVVC2VMYGRQ0XWMRNXQ0BSS01UCWC


## 2 
## 2.1 Data Requirements
Geodata for current residence in Singapore with venues established using Foursquare.
List of Manhattan (MH) neighborhoods with clustered venues established via Foursquare (as in Course  Lab). https://en.wikipedia.org/wiki/List_of_Manhattan_neighborhoods#Midtown_neighborhoods
List of subway metro stations in Manhattan with addresses and geo data (lat,long): https://  en.wikipedia.org/wiki/List_of_New_York_City_Subway_stations_in_Manhattan) , (https://www.google.com/  maps/search/manhattan+subway+metro+stations/@40.7837297,-74.1033043,11z/data=!3m1!4b1)
List of apartments for rent in Manhattan area with information on neighborhood location, address,  number of beds, area size, monthly rent price and complemented with geo data via Nominatim.	http://  www.rentmanhattan.com/index.cfm?page=search&state=results https://www.nestpick.com/search?  city=new-
Place to work in Manhattan (Park Avenue and 53rd St) for reference

## 2.2	Data Sources, Data Processing and Tools used
Singapore data and map is to be created with use of Nominatim , Foursquare and Folium mapping
Manhattan neighborhoods were obtained from Wikipedia and organized by Neighborhoods with geodata  via Nominatim for mapping with Folium.
List of Subway stations was obtained via Wikipedia, NY Transit web site and Google map,
List of apartments for rent was consolidated from web-scraping real estate sites for MH. The geolocation  (lat,long) data was found with algorithm coding and using Nominatim.
Folium map was the basis of mapping with various features to consolidate all data in ONE map where  one can visualize all details needed to make a selection of apartment

## Data according to venues around Neighborhood

In [4]:
address = 'Mccallum Street, Singapore'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore home are {}, {}.'.format(latitude, longitude))

C:\Users\rahul\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Singapore home are 1.2787941, 103.848846.


In [5]:
neighborhood_latitude=1.2792655
neighborhood_longitude=103.8480938

In [6]:
LIMIT = 100 
radius = 500 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=YASVWXRHKI3UTPILR3URBA2BJYJKGKCRYIEABDUXCSI5K5IF&client_secret=32VFUXYBPBYD2MKLBMEMVVC2VMYGRQ0XWMRNXQ0BSS01UCWC&v=20180604&ll=1.2792655,103.8480938&radius=500&limit=100'

In [11]:
results = requests.get(url).json()

In [12]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:

venues = results['response']['groups'][0]['items']
    
SGnearby_venues = json_normalize(venues) 
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]

SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)

SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.head(10)

,name,categories,lat,lng
0,Napoleon Food & Wine Bar,Wine Bar,1.279925,103.847333
1,Native,Cocktail Bar,1.280135,103.846844
2,Mellower Coffee,Café,1.277814,103.848188
3,Muchachos,Burrito Place,1.279072,103.847026
4,Park Bench Deli,Deli / Bodega,1.279872,103.847287
5,Pepper Bowl,Asian Restaurant,1.279371,103.846710
6,Matt's | The Chocolate Shop,Dessert Shop,1.280462,103.846950
7,Freehouse,Beer Garden,1.281254,103.848513
8,PS.Cafe,Café,1.280468,103.846264
9,Sofitel So Singapore,Hotel,1.280124,103.849867


In [14]:
map_sg = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_sg)  
    
map_sg

## Similarly exract the content for rest of the required data set

In [15]:
manhattan_data  = pd.read_csv('mh_neigh_data.csv') 
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
0,Manhattan,Marble Hill,40.876551,-73.910660,2
1,Manhattan,Chinatown,40.715618,-73.994279,2
2,Manhattan,Washington Heights,40.851903,-73.936900,4
3,Manhattan,Inwood,40.867684,-73.921210,3
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0


In [16]:
mh_rent=pd.read_csv('MH_flats_price.csv')
mh_rent.head()

,Address,Area,Price_per_ft2,Rooms,Area-ft2,Rent_Price,Lat,Long
0,West 105th Street,Upper West Side,2.94,5.0,3400,10000,NaN,NaN
1,East 97th Street,Upper East Side,3.57,3.0,2100,7500,NaN,NaN
2,West 105th Street,Upper West Side,1.89,4.0,2800,5300,NaN,NaN
3,CARMINE ST.,West Village,3.03,2.0,1650,5000,NaN,NaN
4,171 W 23RD ST.,Chelsea,3.45,2.0,1450,5000,NaN,NaN


In [17]:
mh=pd.read_csv('NYC_subway_list.csv')
mh.head()

,sub_station,sub_address
0,Dyckman Street Subway Station,"170 Nagle Ave, New York, NY 10034, USA"
1,57 Street Subway Station,"New York, NY 10106, USA"
2,Broad St,"New York, NY 10005, USA"
3,175 Street Station,"807 W 177th St, New York, NY 10033, USA"
4,5 Av and 53 St,"New York, NY 10022, USA"


## merge all the above dataset and will do the analysis in next week .